<a href="https://colab.research.google.com/github/3bdillahiomar/3bdillahiomar/blob/main/00_Beledweyne_Flood_Somalia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[geemap](https://github.com/giswqs/geemap) is an open-source Python package that comes with many helpful features that help you use Earth Engine effectively in Python.

It comes with a function that can help you translate your javascript earth engine code to Python automatically.

Google Colab doesn't come pre-installed with the package, so we install it via pip.

In [1]:
try:
    import geemap
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print('geemap not found, installing via pip in Google Colab...')
        !pip install geemap --quiet
        import geemap
    else:
        print('geemap not found, please install via conda in your environment')

In [2]:
import geemap

In [3]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=NM-HMgZfSW68bAGsN3VEZRBhFU0Ux0apzzAYszx8v6A&tc=-NgTQm6rxStSpWeqyv7O7xKZmhpXcPRpLhMRXcdjJRE&cc=vG6rZlcYCaQ1M1TBb0Bj_gXKjWTVnXdKX21-25ewj3w

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AeaYSHCjvYhhNfYtRdr4j_l2hakS_UCvhxGVwqPJKIS8mVvaWq4nw7FRA2s

Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [4]:
import ee

s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241])

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
  .filter(ee.Filter.date('2023-01-01', '2023-09-01')) \
  .filter(ee.Filter.bounds(geometry)) \
  .filter(ee.Filter.eq('SPACECRAFT_NAME', 'Sentinel-2A' ))

print(filtered.size().getInfo())


10


The automatic conversion of code is done by calling the `geemap.js_snippet_to_py()` function. We first create a string with the javascript code.

In [6]:
javascript_code = """

/*===========================================================================================
                       SAR-FLOOD MAPPING USING A CHANGE DETECTION APPROACH
  ===========================================================================================
  Within this script SAR Sentinel-1 is being used to generate a flood extent map. A change
  detection approach was chosen, where a before- and after-flood event image will be compared.
  Sentinel-1 GRD imagery is being used. Ground Range Detected imagery includes the following
  preprocessing steps: Thermal-Noise Removal, Radiometric calibration, Terrain-correction
  hence only a Speckle filter needs to be applied in the preprocessing.

  ===========================================================================================



   --> Remove the comment-symbol (//) below so Earth Engine recognizes the polygon.*/

var geometry = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
  .filter(ee.Filter.eq('country_na', 'Somalia'));
Map.addLayer(geometry, {color: '00000000'}, 'Somalia');
/* Now hit Run to start the demo!
   Do not forget to delete/outcomment this geometry before creating a new one!
  :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

  *******************************************************************************************
                                  SELECT YOUR OWN STUDY AREA

   Use the polygon-tool in the top left corner of the map pane to draw the shape of your
   study area. Single clicks add vertices, double-clicking completes the polygon.
   **CAREFUL**: Under 'Geometry Imports' (top left in map panel) uncheck the
               geometry box, so it does not block the view on the imagery later.


  *******************************************************************************************
                                       SET TIME FRAME

   Set start and end dates of a period BEFORE the flood. Make sure it is long enough for
   Sentinel-1 to acquire an image (repitition rate = 6 days). Adjust these parameters, if
   your ImageCollections (see Console) do not contain any elements.*/

var before_start= '2023-03-03';
var before_end='2023-03-12';

// Now set the same parameters for AFTER the flood.
var after_start='2023-03-19';
var after_end='2023-03-29';

/********************************************************************************************
                           SET SAR PARAMETERS (can be left default)*/

var polarization = "VH"; /*or 'VV' --> VH mostly is the prefered polarization for flood mapping.
                           However, it always depends on your study area, you can select 'VV'
                           as well.*/
var pass_direction = "DESCENDING"; /* or 'ASCENDING'when images are being compared use only one
                           pass direction. Consider changing this parameter, if your image
                           collection is empty. In some areas more Ascending images exist than
                           than descending or the other way around.*/
var difference_threshold = 1.25; /*threshodl to be applied on the difference image (after flood
                           - before flood). It has been chosen by trial and error. In case your
                           flood extent result shows many false-positive or negative signals,
                           consider changing it! */
//var relative_orbit = 79;
                          /*if you know the relative orbit for your study area, you can filter
                           you image collection by it here, to avoid errors caused by comparing
                           different relative orbits.*/


/********************************************************************************************
  ---->>> DO NOT EDIT THE SCRIPT PAST THIS POINT! (unless you know what you are doing) <<<---
  ------------------>>> now hit the'RUN' at the top of the script! <<<-----------------------
  -----> The final flood product will be ready for download on the right (under tasks) <-----

  ******************************************************************************************/

//---------------------------------- Translating User Inputs ------------------------------//

//------------------------------- DATA SELECTION & PREPROCESSING --------------------------//

// rename selected geometry feature
var aoi = ee.FeatureCollection(geometry);

// Load and filter Sentinel-1 GRD data by predefined parameters
var collection= ee.ImageCollection('COPERNICUS/S1_GRD')
  .filter(ee.Filter.eq('instrumentMode','IW'))
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization))
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction))
  .filter(ee.Filter.eq('resolution_meters',10))
  //.filter(ee.Filter.eq('relativeOrbitNumber_start',relative_orbit ))
  .filterBounds(aoi)
  .select(polarization);

// Select images by predefined dates
var before_collection = collection.filterDate(before_start, before_end);
var after_collection = collection.filterDate(after_start,after_end);

// Print selected tiles to the console

      // Extract date from meta data
      function dates(imgcol){
        var range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]);
        var printed = ee.String('from ')
          .cat(ee.Date(range.get('min')).format('YYYY-MM-dd'))
          .cat(' to ')
          .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'));
        return printed;
      }
      // print dates of before images to console
      var before_count = before_collection.size();
      print(ee.String('Tiles selected: Before Flood ').cat('(').cat(before_count).cat(')'),
        dates(before_collection), before_collection);

      // print dates of after images to console
      var after_count = before_collection.size();
      print(ee.String('Tiles selected: After Flood ').cat('(').cat(after_count).cat(')'),
        dates(after_collection), after_collection);

// Create a mosaic of selected tiles and clip to study area
var before = before_collection.mosaic().clip(aoi);
var after = after_collection.mosaic().clip(aoi);

// Apply reduce the radar speckle by smoothing
var smoothing_radius = 50;
var before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters');
var after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters');


//------------------------------- FLOOD EXTENT CALCULATION -------------------------------//

// Calculate the difference between the before and after images
var difference = after_filtered.divide(before_filtered);

// Apply the predefined difference-threshold and create the flood extent mask
var threshold = difference_threshold;
var difference_binary = difference.gt(threshold);

// Refine flood result using additional datasets

      // Include JRC layer on surface water seasonality to mask flood pixels from areas
      // of "permanent" water (where there is water > 10 months of the year)
      var swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality');
      var swater_mask = swater.gte(10).updateMask(swater.gte(10));

      //Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
      var flooded_mask = difference_binary.where(swater_mask,0);
      // final flooded area without pixels in perennial waterbodies
      var flooded = flooded_mask.updateMask(flooded_mask);

      // Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
      // This operation reduces noise of the flood extent product
      var connections = flooded.connectedPixelCount();
      var flooded = flooded.updateMask(connections.gte(8));

      // Mask out areas with more than 5 percent slope using a Digital Elevation Model
      var DEM = ee.Image('WWF/HydroSHEDS/03VFDEM');
      var terrain = ee.Algorithms.Terrain(DEM);
      var slope = terrain.select('slope');
      var flooded = flooded.updateMask(slope.lt(5));

// Calculate flood extent area
// Create a raster layer containing the area information of each pixel
var flood_pixelarea = flooded.select(polarization)
  .multiply(ee.Image.pixelArea());

// Sum the areas of flooded pixels
// default is set to 'bestEffort: true' in order to reduce compuation time, for a more
// accurate result set bestEffort to false and increase 'maxPixels'.
var flood_stats = flood_pixelarea.reduceRegion({
  reducer: ee.Reducer.sum(),
  geometry: aoi,
  scale: 10, // native resolution
  //maxPixels: 1e9,
  bestEffort: true
  });

// Convert the flood extent to hectares (area calculations are originally given in meters)
var flood_area_ha = flood_stats
  .getNumber(polarization)
  .divide(10000)
  .round();
// //------------------------------  DISPLAY PRODUCTS  ----------------------------------//
// Before and after flood SAR mosaic
Map.centerObject(aoi,8);
Map.addLayer(before_filtered, {min:-25,max:0}, 'Before Flood',0);
Map.addLayer(after_filtered, {min:-25,max:0}, 'After Flood',1);

// Difference layer
Map.addLayer(difference,{min:0,max:2},"Difference Layer",0);

// Flooded areas
Map.addLayer(flooded,{palette:"0000FF"},'Flooded areas');

  //------------------------------------- EXPORTS ------------------------------------//
// Export flooded area as TIFF file
Export.image.toDrive({
  image: flooded,
  description: 'Flood_extent_raster',
  fileNamePrefix: 'flooded',
  region: aoi,
  maxPixels: 1e10
});

// Export flooded area as shapefile (for further analysis in e.g. QGIS)
// Convert flood raster to polygons
var flooded_vec = flooded.reduceToVectors({
  scale: 10,
  geometryType:'polygon',
  geometry: aoi,
  eightConnected: false,
  bestEffort:true,
  tileScale:2,
});

// Export flood polygons as shape-file
Export.table.toDrive({
  collection:flooded_vec,
  description:'Flood_extent_vector',
  fileFormat:'SHP',
  fileNamePrefix:'flooded_vec'
});


////// END OF FLOOD MAPPING
////// Abdillahi Osman Omar

"""

In [7]:
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

import ee
import geemap

m = geemap.Map()

#===========================================================================================
                       SAR-FLOOD MAPPING USING A CHANGE DETECTION APPROACH
  Within this script SAR Sentinel-1 is being used to generate a flood extent map. A change
  detection approach was chosen, where a before- and after-flood event image will be compared.
  Sentinel-1 GRD imagery is being used. Ground Range Detected imagery includes the following
  'preprocessing steps': Thermal-Noise Removal, Radiometric calibration, Terrain-correction
  hence only a Speckle filter needs to be applied in the preprocessing.


   --> Remove the comment-symbol (#) below so Earth Engine recognizes the polygon.#

geometry = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
  .filter(ee.Filter.eq('country_na', 'Somalia'))
m.addLayer(geometry, {'color': '00000000'}, 'Somalia')
# Now hit Run to start the demo!
   Do not forget to delete/outcomment this geometry before cre

The automatic conversion works great. Review it and paste it to the cell below.

In [13]:
import ee
import geemap

m = geemap.Map()
'''
#===========================================================================================
                       SAR-FLOOD MAPPING USING A CHANGE DETECTION APPROACH
  ===========================================================================================
  Within this script SAR Sentinel-1 is being used to generate a flood extent map. A change
  detection approach was chosen, where a before- and after-flood event image will be compared.
  Sentinel-1 GRD imagery is being used. Ground Range Detected imagery includes the following
  'preprocessing steps': Thermal-Noise Removal, Radiometric calibration, Terrain-correction
  hence only a Speckle filter needs to be applied in the preprocessing.

  ===========================================================================================
  '''

#   --> Remove the comment-symbol (#) below so Earth Engine recognizes the polygon.#

geometry = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
  .filter(ee.Filter.eq('country_na', 'Somalia'))
m.addLayer(geometry, {'color': 'blue'}, 'Somalia')
m



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [19]:
before_start= '2023-11-10'
before_end='2023-11-30'

# Now set the same parameters for AFTER the flood.
after_start='2023-12-01'
after_end='2023-12-15'

'''
*******************************************************************************************
                           SET SAR PARAMETERS (can be left default)
'''

polarization = "VH"; #or 'VV' --> VH mostly is the prefered polarization for flood mapping.
                           #However, it always depends on your study area, you can select 'VV'
                           #as well.#
pass_direction = "DESCENDING"; # or 'ASCENDING'when images are being compared use only one
                           #pass direction. Consider changing this parameter, if your image
                           #collection is empty. In some areas more Ascending images exist than
                           #than descending or the other way around.#
difference_threshold = 1.25; #threshodl to be applied on the difference image (after flood
                           #- before flood). It has been chosen by trial and error. In case your
                          # flood extent result shows many False-positive or negative signals,
                          #consider changing it! #
#relative_orbit = 79
                          #if you know the relative orbit for your study area, you can filter
                          #you image collection by it here, to avoid errors caused by comparing
                          # different relative orbits.#

#*******************************************************************************************



print(before_end, before_start)
print(after_end, after_start)

2023-11-30 2023-11-10
2023-12-15 2023-12-01


In [22]:
'''
DO NOT EDIT THE SCRIPT PAST THIS POINT! (unless you know what you are doing)
now hit the'RUN' at the top of the script!
The final flood product will be ready for download on the right (under tasks)

'''

##########################################################################################

#---------------------------------- Translating User Inputs ------------------------------#

#------------------------------- DATA SELECTION & PREPROCESSING --------------------------#

# rename selected geometry feature
aoi = ee.FeatureCollection(geometry)

# Load and filter Sentinel-1 GRD data by predefined parameters
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(aoi) \
  .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)

# Print selected tiles to the console

# Extract date from meta data
def dates(imgcol):
  range = imgcol.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
  printed = ee.String('from ') \
    .cat(ee.Date(range.get('min')).format('YYYY-MM-dd')) \
    .cat(' to ') \
    .cat(ee.Date(range.get('max')).format('YYYY-MM-dd'))
  return printed

# print dates of before images to console
before_count = before_collection.size()
print(ee.String('Tiles selected: Before Flood ').cat('(').cat(before_count).cat(')'),
      dates(before_collection), before_collection)

# print dates of after images to console
after_count = after_collection.size()
print(ee.String('Tiles selected: After Flood ').cat('(').cat(after_count).cat(')'),
      dates(after_collection), after_collection)

# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

#print(after_filtered)

ee.String({
  "functionInvocationValue": {
    "functionName": "String.cat",
    "arguments": {
      "string1": {
        "functionInvocationValue": {
          "functionName": "String.cat",
          "arguments": {
            "string1": {
              "functionInvocationValue": {
                "functionName": "String.cat",
                "arguments": {
                  "string1": {
                    "constantValue": "Tiles selected: Before Flood "
                  },
                  "string2": {
                    "constantValue": "("
                  }
                }
              }
            },
            "string2": {
              "functionInvocationValue": {
                "functionName": "Collection.size",
                "arguments": {
                  "collection": {
                    "functionInvocationValue": {
                      "functionName": "Collection.filter",
                      "arguments": {
                        "collection": {
       

In [24]:
#------------------------------- FLOOD EXTENT CALCULATION -------------------------------#

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets
# Include JRC layer on surface water seasonality to mask flood pixels from areas
# of "permanent" water (where there is water > 10 months of the year)
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))

# Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask, 0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

# Calculate flood extent area
# Create a raster layer containing the area information of each pixel
flood_pixelarea = flooded.select(polarization).multiply(ee.Image.pixelArea())

# Sum the areas of flooded pixels
# default is set to 'bestEffort: True' in order to reduce computation time, for a more
# accurate result set bestEffort to False and increase 'maxPixels'.
flood_stats = flood_pixelarea.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': aoi,
    'scale': 10,  # native resolution
    # maxPixels: 1e9,
    'bestEffort': True
})

# Convert the flood extent to hectares (area calculations are originally given in meters)
flood_area_ha = flood_stats.getNumber(polarization).divide(10000).round()

# #------------------------------  DISPLAY PRODUCTS  ----------------------------------#
# Before and after flood SAR mosaic
m.centerObject(aoi, 8)
m.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Flood', 0)
m.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Flood', 1)

# Difference layer
m.addLayer(difference, {'min': 0, 'max': 2}, "Difference Layer", 0)

# Flooded areas
m.addLayer(flooded, {'palette': "0000FF"}, 'Flooded areas')

m


Map(bottom=490.0, center=[6.047019884725602, 45.8446648053698], controls=(WidgetControl(options=['position', '…

In [28]:
# Export flooded area as TIFF file
task_flooded_tiff = ee.batch.Export.image.toDrive(
    image=flooded,
    description='Flood_extent_raster',
    folder='earthengine',  # Specify your folder in Google Drive
    fileNamePrefix='flooded',
    region=aoi.geometry(),
    scale=10,
    maxPixels=1e10
)
task_flooded_tiff.start()

# Convert flood raster to polygons
flooded_vec = flooded.reduceToVectors(
    scale=10,
    geometryType='polygon',
    geometry=aoi.geometry(),
    eightConnected=False,
    bestEffort=True,
    tileScale=2
)

# Export flooded area as shapefile
task_flooded_shapefile = ee.batch.Export.table.toDrive(
    collection=flooded_vec,
    description='Flood_extent_vector',
    folder='earthengine',  # Specify your folder in Google Drive
    fileFormat='SHP',
    fileNamePrefix='flooded_vec'
)
task_flooded_shapefile.start()


In [30]:
# Now the task will start on your assigned Google Earth Engine account

### Exercise

Take the Javascript code snippet below and use `geemap` to automatically convert it to Python.

---

```
var admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2");

var karnataka = admin2.filter(ee.Filter.eq('ADM1_NAME', 'Karnataka'))

var visParams = {color: 'red'}
Map.centerObject(karnataka)
Map.addLayer(karnataka, visParams, 'Karnataka Districts')
```
---

In [ ]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Create a Map instance
Map = geemap.Map()

# Load the administrative boundaries dataset
admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Filter the dataset for Woqooyi Galbeed
Woqooyi_Galbeed = admin2.filter(ee.Filter.eq('ADM1_NAME', 'Woqooyi Galbeed'))

# Visualization parameters
visParams = {'color': 'red'}

# Center the map on Woqooyi_Galbeed and add the filtered layer
Map.centerObject(Woqooyi_Galbeed)
Map.addLayer(Woqooyi_Galbeed, visParams, 'Woqooyi Galbeed Districts')

# Display the map
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…